# Reiforcment learning UR-3
Setup step

In [27]:
import tensorflow as tf
import numpy as np
import tf_agents
import matplotlib.pyplot as plt

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.policies import py_tf_eager_policy
from tf_agents.drivers import py_driver
from tf_agents.agents.reinforce import reinforce_agent
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step
from tf_agents.specs import tensor_spec
from tf_agents.utils import common
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.drivers import dynamic_step_driver

from tf_env.UR_ENV import UR_env

# Neural Network Policies

In [28]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

n_inputs = 6

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(36, activation="elu", input_shape=[n_inputs]),
    tf.keras.layers.Dense(12, activation="relu"),
])

In [29]:
# size_iteration=1000
# environment = UR_env()

# observation = environment.reset()
# optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
# loss_fn = tf.keras.losses.MSE

# for iteration in range(size_iteration):
    
    
#     # if angle < 0, we want proba(left) = 1., or else proba(left) = 0.
#     target_probas = np.array([([1.] if obs[2] < 0 else [0.])
#                               for obs in observations])
#     with tf.GradientTape() as tape:
#         left_probas = model(np.array(observations))
#         loss = tf.reduce_mean(loss_fn(target_probas, left_probas))
#     print("\rIteration: {}, Loss: {:.3f}".format(iteration, loss.numpy()), end="")
#     grads = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(grads, model.trainable_variables))
#     actions = (np.random.rand(n_environments, 1) > left_probas.numpy()).astype(np.int32)
#     for env_index, env in enumerate(envs):
#         obs, reward, done, info = env.step(actions[env_index][0])
#         observations[env_index] = obs if not done else env.reset()


# REUNFORCE algorithms
REINFORCE is Reward Incremenent = nonnegative Factor * Offset Reinforcment * Characteristic Eligibility.
Based on PG strategy.

# Actor Network Policies

## Giperparameters

In [30]:
number_iterations = 100 # @param {type:"integer"}
fc_layer_params=(50,50,50)
learning_rate = 1e-3 # @param {type:"number"}
number_eval_episodes = 10 # @param {type:"integer"}
collect_max_episodes = 10 # @param {type:"integer"}
collect_max_steps = 20 # @param {type:"integer"}
replay_buffer_capacity = 1000 # @param {type:"integer"}
log_interval = 20 # @param {type:"integer"}
eval_interval = 50 # @param {type:"integer"}


## Environment
Usually we create two environments: one for training and one for evaluation. Most environments are written in pure python, but they can be easily converted to TensorFlow using the TFPyEnvironment wrapper. The original environment's API uses numpy arrays, the TFPyEnvironment converts these to/from Tensors for you to more easily interact with TensorFlow policies and agents.

In [31]:
train_env_py= UR_env()
eval_env_py=UR_env()

train_env_tf=tf_py_environment.TFPyEnvironment(train_env_py)
eval_env_tf=tf_py_environment.TFPyEnvironment(eval_env_py)

In [32]:
train_env_tf.observation_spec()

BoundedTensorSpec(shape=(2, 3), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32))

In [33]:
train_env_tf.action_spec()

BoundedTensorSpec(shape=(6,), dtype=tf.float32, name='action', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32))

## Agent
The algorithm that we use to solve an RL problem is represented as an Agent. In addition to the REINFORCE agent, TF-Agents provides standard implementations of a variety of Agents such as DQN, DDPG, TD3, PPO and SAC.

To create a REINFORCE Agent, we first need an Actor Network that can learn to predict the action given an observation from the environment.

We can easily create an Actor Network using the specs of the observations and actions. We can specify the layers in the network which, in this example, is the fc_layer_params argument set to a tuple of ints representing the sizes of each hidden layer (see the Hyperparameters section above).

In [34]:
from tf_agents.networks import actor_distribution_network

actor_net = actor_distribution_network.ActorDistributionNetwork(
    train_env_tf.observation_spec(),
    train_env_tf.action_spec(),
    fc_layer_params=fc_layer_params)

We also need an optimizer to train the network we just created, and a train_step_counter variable to keep track of how many times the network was updated.

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [36]:
train_step_counter = tf.Variable(0)

In [37]:
tf_agent = reinforce_agent.ReinforceAgent(
    train_env_tf.time_step_spec(),
    train_env_tf.action_spec(),
    actor_network=actor_net,
    optimizer=optimizer,
    normalize_returns=True,
    train_step_counter=train_step_counter)
tf_agent.initialize()

## Policies
In TF-Agents, policies represent the standard notion of policies in RL: given a time_step produce an action or a distribution over actions. The main method is policy_step = policy.action(time_step) where policy_step is a named tuple PolicyStep(action, state, info). The policy_step.action is the action to be applied to the environment, state represents the state for stateful (RNN) policies and info may contain auxiliary information such as log probabilities of the actions.

Agents contain two policies: the main policy that is used for evaluation/deployment (agent.policy) and another policy that is used for data collection (agent.collect_policy).

In [38]:
eval_policy = tf_agent.policy
collect_policy = tf_agent.collect_policy

## Metrics and Evaluation
The most common metric used to evaluate a policy is the average return. The return is the sum of rewards obtained while running a policy in an environment for an episode, and we usually average this over a few episodes. We can compute the average return metric as follows.

In [39]:
def compute_avg_return(
    environment:tf_py_environment.TFPyEnvironment, 
    policy:tf_agents.policies.tf_policy.TFPolicy, 
    num_episodes:int=10
    ):
  
  total_return = 0.0
  for _ in range(num_episodes):
    time_step = environment.reset()
    episode_return = 0.0
    counter_step=0
    counter_step_max=200
    

    # while not time_step.is_last() or counter_step < counter_step_max:
    while counter_step < counter_step_max:
      counter_step+=1
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


## Replay Buffer
Reinforcement learning algorithms use replay buffers to store trajectories of experience when executing a policy in an environment. During training, replay buffers are queried for a subset of the trajectories (either a sequential subset or a sample) to "replay" the agent's experience.

In TF-Agents we use a Driver (see the Driver tutorial for more details) to collect experience in an environment. To use a Driver, we specify an Observer that is a function for the Driver to execute when it receives a trajectory.

TFUniformReplayBuffer is the most commonly used replay buffer in TF-Agents, thus we will use here. In TFUniformReplayBuffer the backing buffer storage is done by tensorflow variables and thus is part of the compute graph.

The buffer stores batches of elements and has a maximum capacity max_length elements per batch segment. Thus, the total buffer capacity is batch_size x max_length elements. The elements stored in the buffer must all have a matching data spec. When the replay buffer is used for data collection, the spec is the agent's collect data spec.

In [65]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    tf_agent.collect_data_spec,
    batch_size=1,
    max_length=1000)

replay_observer = [replay_buffer.add_batch]

# replay_buffer = []
# replay_observer = [replay_buffer.append]

# collect_steps_per_iteration = 10
# collect_op = dynamic_step_driver.DynamicStepDriver(
#   train_env_tf,
#   tf_agent.collect_policy,
#   observers=replay_observer,
#   num_steps=collect_steps_per_iteration).run()

As REINFORCE learns from whole episodes, we define a function to collect an episode using the given data collection policy and save the data (observations, actions, rewards etc.) as trajectories in the replay buffer. Here we are using 'PyDriver' to run the experience collecting loop. 

In [80]:
from tf_agents.environments import PyEnvironment
from tf_agents.policies.py_policy import PyPolicy
from tf_agents.metrics import py_metrics

# def collect_episode(environment:PyEnvironment, policy,max_steps:int=1000, max_episodes:int=1):
#   driver = py_driver.PyDriver(
#     environment,
#     py_tf_eager_policy.PyTFEagerPolicy(
#       policy, use_tf_function=True),
#     replay_observer,
#     max_steps=max_steps,
#     max_episodes=max_episodes)
#   initial_time_step = environment.reset()
#   driver.run(initial_time_step)

def collect_episode(environment, agent,max_steps:int=100, max_episodes:int=1):
  dynamic_step_driver.DynamicStepDriver(
    environment,
    agent.collect_policy,
    observers=replay_observer,
    num_steps=10).run()




collect_episode(train_env_tf,tf_agent)
dataset = replay_buffer.as_dataset(
      sample_batch_size=1)
iterator = iter(dataset)
loss = None
trajectories, _ = next(iterator)
print(trajectories._fields)
print(trajectories.observation)

loss = tf_agent.train(experience=trajectories)
print('Training loss: ', loss.loss.numpy())
# collect_episode(train_env_py,collect_policy,10,5)
# print(replay_buffer)

('step_type', 'observation', 'action', 'policy_info', 'next_step_type', 'reward', 'discount')
tf.Tensor(
[[[-233.47815   -65.37784  -216.97325 ]
  [ -15.516002  121.334    -110.74701 ]]], shape=(1, 2, 3), dtype=float32)


TypeError: 'NoneType' object is not callable

## Training the agent
The training loop involves both collecting data from the environment and optimizing the agent's networks. Along the way, we will occasionally evaluate the agent's policy to see how we are doing.

In [42]:
# (Optional) Optimize by wrapping some of the code in a graph using TF function.
tf_agent.train = common.function(tf_agent.train)

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env_tf, tf_agent.policy, number_eval_episodes)
returns = [avg_return]

# print(returns)

[3.4047084]


In [51]:
for _ in range(number_iterations):

  # Collect a few episodes using collect_policy and save to the replay buffer.
  collect_episode(train_env_py, collect_policy,collect_max_steps, collect_max_episodes)

  # Use data from the buffer and update the agent's network.
  # iterator = iter(replay_buffer.as_dataset(sample_batch_size=1))
  iterator = iter(replay_buffer)

  # print(replay_buffer.as_dataset(sample_batch_size=5))
  trajectories = next(iterator)

  train_loss = tf_agent.train(experience=trajectories)  

  replay_buffer.clear()

  step = tf_agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env_tf, tf_agent.policy, number_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

ValueError: All of the Tensors in `value` must have two outer dimensions. Specifically, tensors must have shape `[B, T] + spec.shape`.
Full shapes of value tensors:
  Trajectory(
{'action': TensorShape([6]),
 'discount': TensorShape([]),
 'next_step_type': TensorShape([]),
 'observation': TensorShape([2, 3]),
 'policy_info': (),
 'reward': TensorShape([]),
 'step_type': TensorShape([])}).
Expected shapes (excluding the two outer dimensions):
  Trajectory(
{'action': TensorShape([6]),
 'discount': TensorShape([]),
 'next_step_type': TensorShape([]),
 'observation': TensorShape([2, 3]),
 'policy_info': (),
 'reward': TensorShape([]),
 'step_type': TensorShape([])}).

## Visualization

In [ ]:
steps = range(0, number_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim(top=250)